In [2]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key

from api_keys import g_key

In [4]:
 # Store csv data into a Pandas df
    
city_info_df = pd.read_csv("data/cities.csv")
city_info_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dingzhou,38.5131,114.9956,48.31,65,100,5.82,CN,1635205439
1,1,rikitea,-23.1203,-134.9692,75.45,72,69,18.92,PF,1635205439
2,2,yellowknife,62.4560,-114.3525,42.42,94,90,1.99,CA,1635205439
3,3,manado,1.4870,124.8455,83.17,83,20,1.14,ID,1635205440
4,4,kaspiysk,42.8816,47.6392,44.78,81,90,11.01,RU,1635205440


In [5]:
# Configure gmaps API key

gmaps.configure(api_key=g_key)

In [9]:
# Humidity heatmap

humidity = city_info_df["Humidity"]
locations = city_info_df[["Lat", "Lng"]]
fig_1 = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig_1.add_layer(heat_layer)

fig_1

Figure(layout=FigureLayout(height='420px'))

In [14]:
 # Narrow down cities based on criteria and drop any null values
    
city_info_filtered_df = city_info_df.loc[(city_info_df["Max Temp"] < 90) & (city_info_df["Max Temp"] > 75) \
                                    & (city_info_df["Cloudiness"] == 0) \
                                    & (city_info_df["Wind Speed"] < 10)].dropna()
city_info_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,catamarca,-28.4696,-65.7852,80.04,21,0,9.98,AR,1635205462
180,180,umm lajj,25.0213,37.2685,78.98,52,0,5.79,SA,1635205420
200,200,gwadar,25.1216,62.3254,76.80,73,0,1.81,PK,1635205519
264,264,bilma,18.6853,12.9164,80.64,16,0,4.34,NE,1635205386
511,511,santa rosalia,27.3167,-112.2833,80.49,42,0,4.18,MX,1635205669


In [15]:
# Create DataFrame (hotels_df) to create empty column to store hotel names along with their location information

hotels_df = city_info_filtered_df[["City", "Country", "Lat", "Lng"]].copy()
hotels_df["Hotel Name"] = ""
hotels_df

,City,Country,Lat,Lng,Hotel Name
60,catamarca,AR,-28.4696,-65.7852,
180,umm lajj,SA,25.0213,37.2685,
200,gwadar,PK,25.1216,62.3254,
264,bilma,NE,18.6853,12.9164,
511,santa rosalia,MX,27.3167,-112.2833,


In [16]:
# search for a hotel based on specific radius

parameters = {
    "radius": 3000,
    "types": "lodging",
    "key": g_key
}

# Run through df

for index, row in hotels_df.iterrows():
    
    # Pull the latitude and longitutde from the df
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # F string including the latitude and the longitude
    
    parameters["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    
    url_address = requests.get(base_url, params=parameters)
    
    # convert to json
    url_address = url_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotels_df.loc[index, "Hotel Name"] = url_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotels_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
60,catamarca,AR,-28.4696,-65.7852,
180,umm lajj,SA,25.0213,37.2685,
200,gwadar,PK,25.1216,62.3254,
264,bilma,NE,18.6853,12.9164,
511,santa rosalia,MX,27.3167,-112.2833,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotels_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotels_info)
fig_1.add_layer(markers)

# Display figure

fig_1

Figure(layout=FigureLayout(height='420px'))